# RNN with LSTM with Own Dataset

In this lab one will use own CSV text dataset for training a simple RNN for sentiment classification (here: a binary classification problem with two labels, positive and negative) using LSTM (Long Short Term Memory) cells and GRU Cells.

In [280]:
import torch
import torch.nn.functional as F
from torchtext import data
from torchtext  import datasets
import time
import random
import pandas as pd
import gzip
import torchtext
import spacy

torch.backends.cudnn.deterministic = True

## General Settings

In [281]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 1e-4
BATCH_SIZE = 100
NUM_EPOCHS = 15
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 100 #At minimum 128 - needs to contain all the words
HIDDEN_DIM = 275 #Original 256
OUTPUT_DIM = 1

## Dataset

The following cells will download the IMDB movie review dataset (http://ai.stanford.edu/~amaas/data/sentiment/) for positive-negative sentiment classification in as CSV-formatted file:

In [282]:
!gunzip -f movie_data.csv.gz 

'gunzip' is not recognized as an internal or external command,
operable program or batch file.


Check that the dataset looks okay:

In [283]:
df = pd.read_csv('movie_data.csv')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [284]:
del df

Define the Label and Text field formatters:

In [285]:
TEXT = torchtext.legacy.data.Field(sequential=True,
                  tokenize='spacy',
                  include_lengths=True) # necessary for packed_padded_sequence

LABEL = torchtext.legacy.data.LabelField(dtype=torch.float)

D:\Anaconda\lib\site-packages\torchtext\data\utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


Process the dataset:

In [286]:
fields = [('review', TEXT), ('sentiment', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
    path="movie_data.csv", format='csv',
    skip_header=True, fields=fields)

Split the dataset into training, validation, and test partitions:

In [287]:
train_data, valid_data, test_data = dataset.split(
    split_ratio=[0.75, 0.05, 0.2],
    random_state=random.seed(RANDOM_SEED))
#One may want to vary the test, train split percentages
print(f'Num Train: {len(train_data)}')
print(f'Num Valid: {len(valid_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 37500
Num Valid: 10000
Num Test: 2500


Build the vocabulary based on the top "VOCABULARY_SIZE" words:

In [288]:
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 2


In [289]:
LABEL.vocab.freqs

Counter({'0': 18742, '1': 18758})

The TEXT.vocab dictionary will contain the word counts and indices. The reason why the number of words is VOCABULARY_SIZE + 2 is that it contains to special tokens for padding and unknown words: `<unk>` and `<pad>`.

Make dataset iterators:

In [290]:
train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    sort_within_batch=True, # necessary for packed_padded_sequence
    sort_key=lambda x: len(x.review),
    device=DEVICE)

Testing the iterators (note that the number of rows depends on the longest document in the respective batch):

In [291]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.review[0].size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break
    
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.review[0].size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.review[0].size()}')
    print(f'Target vector size: {batch.sentiment.size()}')
    break

Train
Text matrix size: torch.Size([518, 100])
Target vector size: torch.Size([100])

Valid:
Text matrix size: torch.Size([50, 100])
Target vector size: torch.Size([100])

Test:
Text matrix size: torch.Size([68, 100])
Target vector size: torch.Size([100])


## Model

 ### The primary goal of this lab is to vary the hyperparameters of the LSTM model and see the results and provide analysis
 ### The second task is to use a another RNN cell such as GRU and perform parameter tuning and report the results.
 
 ### The remainder of the code will have to be modified accordingly. 

In [292]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        #Here is a preliminary model using LSTM cell
        #The primary goal of this lab is to vary the dimensions of the embeddings and see the results
        #The second task is to use a another RNN cell such as GRU and perform parameter tuning and report the results.
        
        #Xt -> Ht
        #Ht-> yt (output)
        #Repeat again and again
        #t time steps = a lot more parameters
        
        #Convert Data -> Matrix
        self.embedding = nn.Embedding(input_dim, embedding_dim) #Convert Input Data (bag of words)-> Matrix Format (with dimensions = embedding_dim)
        
        #Use RNN on embedding dimension (128 dim) -> Hidden Layers (hidden_dim - 256 dim) 
        #Look at nn.LSTM hyperparameter documentation GOAL 1
        #Change nn.LSTM to nn.GRU (difference: number of parameters/architecture of network vary abit) GOAL 2
        self.rnn = nn.GRU(embedding_dim, hidden_dim) #Change LSTM to GRU
        
        #Get output from hidden layer (scalar output)
        self.fc = nn.Linear(hidden_dim, output_dim) #Desired output is binary format (integer): Dimesnion = 1
        
    def forward(self, text, text_length):

        #[sentence len, batch size] => [sentence len, batch size, embedding size]
        embedded = self.embedding(text)
        
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length)
        
        #[sentence len, batch size, embedding size] => 
        #  output: [sentence len, batch size, hidden size]
        #  hidden: [1, batch size, hidden size]
        
        #packed_output, (hidden, cell) = self.rnn(packed) #LSTM output = output, (h0,c0)
        packed_output, hidden = self.rnn(packed) #GRU output = output, h0
        
        return self.fc(hidden.squeeze(0)).view(-1) #Returns Integer (0 or 1)

In [293]:
INPUT_DIM = len(TEXT.vocab)

torch.manual_seed(RANDOM_SEED)
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

## Training

In [294]:
#Know accuracy of model
#Loss fxc 
def compute_binary_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            text, text_lengths = batch_data.review
            logits = model(text, text_lengths.cpu())
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.sentiment.size(0)
            correct_pred += (predicted_labels.long() == batch_data.sentiment.long()).sum()
        return correct_pred.float()/num_examples * 100

In [295]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text, text_lengths = batch_data.review
        
        ### FORWARD AND BACK PROP
        logits = model(text, text_lengths.cpu())
        cost = F.binary_cross_entropy_with_logits(logits, batch_data.sentiment)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Cost: {cost:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_binary_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_binary_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_binary_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/015 | Batch 000/375 | Cost: 0.6933
Epoch: 001/015 | Batch 050/375 | Cost: 0.6833
Epoch: 001/015 | Batch 100/375 | Cost: 0.6690
Epoch: 001/015 | Batch 150/375 | Cost: 0.6693
Epoch: 001/015 | Batch 200/375 | Cost: 0.6421
Epoch: 001/015 | Batch 250/375 | Cost: 0.6222
Epoch: 001/015 | Batch 300/375 | Cost: 0.6382
Epoch: 001/015 | Batch 350/375 | Cost: 0.5222
training accuracy: 69.01%
valid accuracy: 68.59%
Time elapsed: 0.56 min
Epoch: 002/015 | Batch 000/375 | Cost: 0.6549
Epoch: 002/015 | Batch 050/375 | Cost: 0.5177
Epoch: 002/015 | Batch 100/375 | Cost: 0.5493
Epoch: 002/015 | Batch 150/375 | Cost: 0.5976
Epoch: 002/015 | Batch 200/375 | Cost: 0.5151
Epoch: 002/015 | Batch 250/375 | Cost: 0.5484
Epoch: 002/015 | Batch 300/375 | Cost: 0.4513
Epoch: 002/015 | Batch 350/375 | Cost: 0.5113
training accuracy: 78.39%
valid accuracy: 77.89%
Time elapsed: 1.12 min
Epoch: 003/015 | Batch 000/375 | Cost: 0.4780
Epoch: 003/015 | Batch 050/375 | Cost: 0.4220
Epoch: 003/015 | Batch 100/3

In [296]:
import spacy
nlp = spacy.load("en_core_web_sm")

def predict_sentiment(model, sentence):
    # based on:
    # https://github.com/bentrevett/pytorch-sentiment-analysis/blob/
    # master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [297]:
print('Probability positive:')
1-predict_sentiment(model, "This is such an awesome movie, I really love it!")

Probability positive:


0.7943425476551056

In [298]:
print('Probability negative:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")

Probability negative:


0.8739234805107117

- With Default Settings:  EMBEDDING_DIM = 128,  HIDDEN_DIM = 256, LSTM  
Total Training Time: 6.15 min  
Test accuracy: 84.84%  

***Changing HIDDEN_DIM***  
With Default Settings:  EMBEDDING_DIM = 128, LSTM  
- Changed Settings: HIDDEN_DIM = 64,  
Total Training Time: 3.93 min  
Test accuracy: 86.08%  
- Changed Settings: HIDDEN_DIM = 384,  
Total Training Time: 8.84 min  
Test accuracy: 86.88%  
- Changed Settings: HIDDEN_DIM = 200,  
Total Training Time: 5.67 min  
Test accuracy: 87.36%  
- Changed Settings: HIDDEN_DIM = 225,  
Total Training Time: 5.67 min  
Test accuracy: 87.36%  
  
Seems that the best accuracy comes from HIDDEN_DIM ~ 200

***Changing EMBEDDED_DIM***  
With Settings:  LSTM, HIDDEN_DIM = 200
- Changed Settings: EMBEDDING_DIM = 128,  
Total Training Time: 5.67 min  
Test accuracy: 87.36%  
- Changed Settings: EMBEDDING_DIM = 100,  
Total Training Time: 5.89 min  
Test accuracy: 87.12%  
- Changed Settings: EMBEDDING_DIM = 50,  
Total Training Time: 5.58 min  
Test accuracy: 84.56%  
- Changed Settings: EMBEDDING_DIM = 200,  
Total Training Time: 5.74 min  
Test accuracy: 86.44%  
- Changed Settings: EMBEDDING_DIM = 150,  
Total Training Time: 6.68 min  
Test accuracy: 85.64%  
  
Seems that the best accuracy comes from EMBEDDED_DIM ~ 128

***Changing GRU***  
- With Settings: HIDDEN_DIM = 200, EMBEDDING_DIM = 128,  
Total Training Time: 4.70 min  
Test accuracy: 87.16%  
- With Settings: HIDDEN_DIM = 256, EMBEDDING_DIM = 128,  THIS IS THE DEFAULT SETTING  
Total Training Time: 5.55 min  
Test accuracy: 87.68%  
- With Settings: HIDDEN_DIM = 275, EMBEDDING_DIM = 128, 
Total Training Time: 7.20 min  
Test accuracy: 89.44%  
- With Settings: HIDDEN_DIM = 300, EMBEDDING_DIM = 128,   
Total Training Time: 6.74 min  
Test accuracy: 88.36%  
- With Settings: HIDDEN_DIM = 275, EMBEDDING_DIM = 100,   
Total Training Time: 8.12 min  
Test accuracy: 88.96%  

Seems that the best accuracy we can get is with GRU and HIDDEN_DIM = 275, EMBEDDING_DIM = 128.